In [5]:
import logging
import couchdb
import json
import sys
sys.path.append('./sentiment analysis/release')
import sentiment_prediction as senti
sys.path.append('./topic analysis')
import scorer
sys.path.append('../util')
import text_util
# sys.path.append('../util')
# import MapRegion

In [6]:
'''
    Get the source db(the db created by harverster)
    and the destination db (store the result)
'''
def get_source_db():
    config = None
    db_server = None
    source_db = None
    with open("db_config.json") as cf:
        config = json.load(cf)
        try:
            db_server = couchdb.Server(config["Servers"][0])
            source_db_name = config["DB"][0]
            if source_db_name not in db_server:
                logging.error("Database %s does not exist, something went wrong." %(source_db_name))
                return
            source_db = db_server[source_db_name]
        except Exception as e:
            logging.error("error happened: %s" %e)
            sys.exit(2)
    return source_db

def get_dest_db():
    config = None
    db_server = None
    dest_db = None
    with open("db_config.json") as cf:
        config = json.load(cf)
        try:
            db_server = couchdb.Server(config["Servers"][0])
            dest_db_name = config["DB"][1]
            if dest_db_name in db_server:
                dest_db = db_server[dest_db_name]
        except Exception as e:
            logging.error("error happened: %s" %e)
            sys.exit(2)
    return dest_db

def create_dest_db():
    config = None
    db_server = None
    dest_db = None
    with open("db_config.json") as cf:
        config = json.load(cf)
        try:
            db_server = couchdb.Server(config["Servers"][0])
            dest_db_name = config["DB"][1]
            if dest_db_name in db_server:
                logging.info("db has existed, create a new one")
                db_server.delete(dest_db_name)
                dest_db = db_server.create(dest_db_name)
            else:
                dest_db = db_server.create(dest_db_name)
        except Exception as e:
            logging.error("error happened: %s" %e)
            sys.exit(2)
    return dest_db

'''
    This is the excution part,
    the analysis result will be returned
'''
def analyze(document,senti_analyzer,alcohols_scorer,fastfood_scorer,smoking_scorer):
    created_at = document["created_at"]
    raw_text = document["text"]
    location = document["location"]
    coordinates = document["coordinates"]
    label, proba = senti_analyzer.prediction(raw_text)
    new_text = text_util.preprocess_tweet(raw_text)
    # 0:NEGATIVE 1:POSITIVE
    label_flag = 0 if label=="NEGATIVE" else 1
    ##########add Region here###################
    alcohols_sc = alcohols_scorer.get_score_v2(new_text)
    fastfood_sc = fastfood_scorer.get_score_v2(new_text)
    smoking_sc = smoking_scorer.get_score_v2(new_text)
    return {"created_at":created_at,"text":new_text,"label":label_flag,"probability":proba,
                       "alcohols_score":alcohols_sc,"fastfood_score":fastfood_sc,"smoking_score":smoking_sc}

In [9]:
from mpi4py import MPI
import time

comm = MPI.COMM_WORLD
comm_rank = comm.Get_rank()
comm_size = comm.Get_size()

modelpath = './sentiment analysis/model/sentiment_lstm.h5'
pklpath = './sentiment analysis/model/sentiment140-freqdist.pkl'
senti_analyzer = senti.sentianalyser(modelpath, pklpath, stemmer = False)
start_time = None
end_time = None
if comm_rank == 0:
    start_time = time.time()
    source_db = get_source_db()
    dest_db = create_dest_db()
else:
    source_db = get_source_db()
    dest_db = None
    while dest_db == None:
        dest_db = get_dest_db()
alcohols_dict = "./topic analysis/dictionary/alcohols.txt"
fastfood_dict = "./topic analysis/dictionary/fastfood.txt"
smoking_dict = "./topic analysis/dictionary/smoking.txt"
alcohols_scorer = scorer.Scorer(senti_analyzer, alcohols_dict)
fastfood_scorer = scorer.Scorer(senti_analyzer, fastfood_dict)
smoking_scorer = scorer.Scorer(senti_analyzer, alcohols_dict)
index = 0
buffer = 0
result = []
for ele in source_db:
    index += 1
    print(index)
    if index % comm_size == comm_rank: 
        buffer += 1
        document = source_db[ele]
        result.append(analyze(document,senti_analyzer,alcohols_scorer,fastfood_scorer,smoking_scorer))
        if buffer == 100:
            dest_db.update(result)
            buffer = 0
            result = []
if buffer != 0 and len(result) != 0:
    dest_db.update(result)
print("Analyzing successfully!")
if comm_rank == 0:
    end_time = time.time()
    print("time consumed: %s" %(end_time-start_time))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
